In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import imageio.v2 as imageio
from scipy.spatial import Delaunay, ConvexHull
from scipy.spatial.distance import directed_hausdorff

from scipy import stats
from scipy.special import expit
from shapely.geometry import LineString
import networkx as nx
import plotly.express as px

uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c nfl-big-data-bowl-2024
!unzip nfl-big-data-bowl-2024.zip
!unzip week1_game2022090800.csv.zip

 98% 275M/279M [00:08<00:00, 36.0MB/s]
100% 279M/279M [00:08<00:00, 35.8MB/s]
Archive:  nfl-big-data-bowl-2024.zip
  inflating: games.csv               
  inflating: players.csv             
  inflating: plays.csv               
  inflating: tackles.csv             
  inflating: tracking_week_1.csv     
  inflating: tracking_week_2.csv     
  inflating: tracking_week_3.csv     
  inflating: tracking_week_4.csv     
  inflating: tracking_week_5.csv     
  inflating: tracking_week_6.csv     
  inflating: tracking_week_7.csv     
  inflating: tracking_week_8.csv     
  inflating: tracking_week_9.csv     
unzip:  cannot find or open week1_game2022090800.csv.zip, week1_game2022090800.csv.zip.zip or week1_game2022090800.csv.zip.ZIP.


In [ ]:
# Initialize Data
players=pd.read_csv('players.csv')
week1=pd.read_csv('tracking_week_1.csv')
week2=pd.read_csv('tracking_week_2.csv')
week3=pd.read_csv('tracking_week_3.csv')
week4=pd.read_csv('tracking_week_4.csv')
week5=pd.read_csv('tracking_week_5.csv')
week6=pd.read_csv('tracking_week_6.csv')
week7=pd.read_csv('tracking_week_7.csv')
week8=pd.read_csv('tracking_week_8.csv')
week9=pd.read_csv('tracking_week_9.csv')
games=pd.read_csv('games.csv')
plays=pd.read_csv('plays.csv')
tackles = pd.read_csv('tackles.csv')

In [ ]:
def extract_data_sets(data, gameId, playId, frameId):
    wg = data[(data['gameId'] == gameId)].copy()
    wgp = wg[(wg['playId'] == playId)].copy()

    wgpf = wgp[(wgp['frameId'] == frameId)].copy()
    wgpf_time = wgpf['time'].iloc[0]
    return wg, wgp, wgpf, wgpf_time

def processToVisualizeFrame(df1, df2, df3, df4, df5, playId, time, gameId):
    df = df1[(df1['playId'] == playId) & (df1['time'] == time) & (df1['gameId'] == gameId)].copy()

    df4 = df4.drop(columns=['displayName'])
    df = pd.merge(df, df2, on='gameId', how='left')  # Merges df and games
    df = pd.merge(df, df3, on=['playId', 'gameId'], how='left')  # Merges df and plays
    df = pd.merge(df, df4, on='nflId', how='left')  # Merges df and players
    df = pd.merge(df, df5, on=['playId', 'gameId', 'nflId'], how='left')  # Merges df and tackles

    df.loc[df['playDirection'] == 'right', 'x'] = 120 - df.loc[df['playDirection'] == 'right', 'x']
    df.loc[df['playDirection'] == 'right', 'y'] = 53.3 - df.loc[df['playDirection'] == 'right', 'y']
    df.loc[df['playDirection'] == 'right', 'dir'] += 180
    df['dir'] %= 360
    df.loc[df['playDirection'] == 'right', 'o'] += 180
    df['o'] %= 360

    df['rDir'] = np.deg2rad(df['dir'].astype(float))  # Converts angle in degrees to radians
    df['xComponent'] = np.cos(df['rDir'].astype(float))  # Converts angle into an x component
    df['yComponent'] = np.sin(df['rDir'].astype(float))  # Converts angle into a y component
    df['xSpeed'] = df['xComponent'] * df['s']  # Calculates magnitude of speed by multiplying x component by magnitude of speed
    df['ySpeed'] = df['yComponent'] * df['s']  # Calculates magnitude of speed by multiplying y component by magnitude of speed

    bcx = df.loc[df['displayName'] == df['ballCarrierDisplayName'], 'x'].values[0]  # Retrieves Ball Carrier x coordinate
    bcy = df.loc[df['displayName'] == df['ballCarrierDisplayName'], 'y'].values[0]  # Retrieves Ball Carrier y coordinate

    df['xDist_to_bc'] = bcx - df['x']  # Calculates difference in x between player and ball carrier
    df['yDist_to_bc'] = bcy - df['y']  # Calculates difference in y between player and ball carrier
    df['dist_to_bc'] = np.sqrt(df['xDist_to_bc']**2 + df['yDist_to_bc']**2)  # Calculates distance between player and ball carrier

    return df


In [ ]:
def calculate_path_weights(pdf, start, node):
    x_over_y = (node[0] - start[0]) / (node[1] - start[1])
    iterations = abs(round(node[1] - start[1] * 10))
    path_weights = []

    # Precompute x and y coordinates for the entire range
    x_coordinates = np.array([start[0] * 10 - index * x_over_y for index in range(iterations)])
    y_coordinates = np.round(start[1] * 10) - np.arange(iterations)

    # Use vectorized operations to calculate weights
    for index in range(iterations):
        x_coord = x_coordinates[index]
        y_coord = y_coordinates[index]

        x_floor, x_ceil = int(np.floor(x_coord)) - 10, int(np.ceil(x_coord)) + 10
        y_val = int(y_coord)

        path_weights.append(np.sum(pdf[x_floor:x_ceil, y_val]))

    return np.sum(path_weights) / 5

def adjust_distance(distance, path_weight, start, end):
    adjusted_distance = distance * expit(path_weight)
    return float(adjusted_distance)

def create_Graph(start, df, pdf):
    graph = {}

    if df.empty:
        next_nodes = list([(x / 100, 10) for x in np.arange(15, 5315, 100)])
    else:
        closest_Defender = df[df['x'] == df['x'].max()]
        y = closest_Defender['x'].values[0]
        next_nodes = list([(x / 100, y) for x in np.arange(15, 5315, 100)])

    for node in next_nodes:
        distance = ((start[0] - node[0])**2 + (start[1] - node[1])**2)**0.5
        if isinstance(pdf, float):
            adjusted_distance = distance
        else:
            path_weight = calculate_path_weights(pdf, start, node) / distance
            adjusted_distance = adjust_distance(distance, path_weight, start, node)
        graph[adjusted_distance] = [start, node]
    return graph

def get_Ideal_Path(start, graph):
    best_cost = min(list(graph.keys()))
    best_path_edges = graph[best_cost]

    return best_path_edges

def create_bc_pdf(df, locations):
    # Rotation Matrix
    xComponent, yComponent = df['xComponent'].values[0], df['yComponent'].values[0]
    r_Matrix = np.array([[xComponent, -yComponent], [yComponent, xComponent]])

    # Scaling Matrix
    speed_Ratio = (df['s'].values[0] ** 2) / (13.25 ** 2)
    bc_x = df['y'].values[0] + df['xSpeed'].values[0] * 0.5
    bc_y = df['x'].values[0] + df['ySpeed'].values[0] * 0.5

    dist = 2
    topLeftSMatrix = (dist + dist * speed_Ratio) * 0.5
    botRightSMatrix = (dist - dist * speed_Ratio) * 0.5
    s_Matrix = np.array([[topLeftSMatrix + 0.000001, 0], [0, botRightSMatrix - 0.000001]])

    # Covariance Matrix Calculations
    covariance_matrix = r_Matrix @ s_Matrix @ s_Matrix.T @ np.linalg.inv(r_Matrix)

    # Mu Values
    mu_val_x = df['y'].values[0] + df['xSpeed'].values[0] * 0.2
    mu_val_y = df['x'].values[0] + df['ySpeed'].values[0] * 0.2
    bc_mu = np.array([mu_val_x, mu_val_y])

    # Creating BallCarrier PDF
    bc_pdf = stats.multivariate_normal.pdf(locations, mean=bc_mu, cov=covariance_matrix)

    return bc_pdf

def create_pdf(df, locations, bc, graph=False):
    defensive_pdf, offensive_pdf = 0.0, 0.0
    defenders_pdfs = []

    for index, row in df.iterrows():
        if row['displayName'] == row['ballCarrierDisplayName'] or row['displayName'] == 'football':
            continue

        # Rotation Matrix
        xComponent, yComponent = row['xComponent'], row['yComponent']
        r_Matrix = np.array([[xComponent, -yComponent], [yComponent, xComponent]])
        i_r_Matrix = np.linalg.inv(r_Matrix)

        # Scaling Matrix
        speed_Ratio = (row['s'] ** 2) / (13.25 ** 2)
        topLeftSMatrix = 0.5 + (row['dist_to_bc'] + row['dist_to_bc'] * speed_Ratio) * 0.5
        botRightSMatrix = 0.5 + (row['dist_to_bc'] - row['dist_to_bc'] * speed_Ratio) * 0.5
        s_Matrix = np.array([[topLeftSMatrix + 0.000001, 0], [0, botRightSMatrix - 0.000001]])

        # Covariance Matrix Calculations
        rs_Matrix = np.dot(r_Matrix, s_Matrix)
        rss_Matrix = np.dot(rs_Matrix, s_Matrix)
        covariance_Matrix = np.dot(rss_Matrix, i_r_Matrix)

        # Mu Values
        mu_val_x = row['y'] + row['xSpeed'] * 0.5
        mu_val_y = row['x'] + row['ySpeed'] * 0.5
        mu = [mu_val_x, mu_val_y]

        player_pdf = stats.multivariate_normal(mu, covariance_Matrix).pdf(locations)

        if row['club'] == row['defensiveTeam']:
            if not graph:
                defenders_pdfs.append(player_pdf)
            defensive_pdf += player_pdf
        elif row['club'] == row['possessionTeam']:
            closest_tackler_distance = min(df.loc[df['club'] == df['defensiveTeam']]['dist_to_bc'])

            closest_tackler_distance_blocker = 13105
            yCoordinatePassRusher = 0
            xCoordinatePassRusher = 0
            speedPassRusher = 0

            df_defensive = df.loc[df['club'] == df['defensiveTeam']]
            for _, defense_row in df_defensive.iterrows():
                distance = np.sqrt((defense_row['x'] - row['x']) ** 2 + (defense_row['y'] - row['y']) ** 2)
                if distance <= closest_tackler_distance_blocker:
                    closest_tackler_distance_blocker = distance
                    xCoordinatePassRusher = defense_row['x']
                    yCoordinatePassRusher = defense_row['y']
                    speedPassRusher = defense_row['s']

            p12 = row['dist_to_bc']
            p23 = np.sqrt((bc['x'].values[0] - xCoordinatePassRusher) ** 2 + (bc['y'].values[0] - yCoordinatePassRusher) ** 2)
            p13 = np.sqrt((row['x'] - xCoordinatePassRusher) ** 2 + (row['y'] - yCoordinatePassRusher) ** 2)

            angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))
            degreesAngleBetweenThreePlayers = np.degrees(angleBetweenThreePlayers)
            angleFrom180 = abs(degreesAngleBetweenThreePlayers - 180)

            player_pdf = degreesAngleBetweenThreePlayers * (player_pdf / 180)
            offensive_pdf -= player_pdf

    if not graph:
        return defenders_pdfs, defensive_pdf, offensive_pdf
    else:
        return defensive_pdf, offensive_pdf

COLORMAPS = {
    'offense': 'Blues',
    'defense': 'Reds',
    'combined': 'Purples',
    'bc': 'Greens',
    'pressure': 'Purples'
}

def graph_pdf(ax, x, y, pdf, entity):
    cmap = COLORMAPS.get(entity, None)
    if cmap == 'Blues':
        pdf = -pdf
    divisions = 4
    peak = np.amax(pdf)
    step = peak / divisions
    levels = np.arange(0.0, peak, step) + step
    # levels = [0.005, 0.01, 0.02, 10]
    return ax.contourf(x, y, pdf, levels=levels, cmap=plt.matplotlib.colormaps[cmap], alpha=0.4, zorder=2)


def process(df, pdf):
    bc = df.loc[df['displayName'] == df['ballCarrierDisplayName']]
    dfDefensive = df.loc[(df['club']== df['defensiveTeam']) & (bc['x'].values[0] > (df['x'])) & ((df['x']) > 10)]

    start = tuple(bc[['y','x']].values[0])

    if start[1] <= 10:
        return None

    graph = create_Graph(start, dfDefensive, pdf)
    ideal_path = get_Ideal_Path(start, graph)
    return ideal_path

def process_frame(df, start_frame):
    bc = df.loc[df['displayName'] == df['ballCarrierDisplayName']]
    df_players = df.loc[bc['x'].values[0] + 2.5 > df['x']]

    x, y = np.mgrid[0:53.3:0.1, 0:120:0.1]
    locations = np.dstack((x, y))

    bc_pdf = create_bc_pdf(bc, locations)

    defenders_pdf, defensive_pdf, offensive_pdf = create_pdf(df_players, locations, bc)

    combined_pdf = offensive_pdf + defensive_pdf

    pressure_pdf = combined_pdf * bc_pdf

    path_deviation = 0

    if df['frameId'].values[0] >= start_frame:
        ideal_path = process(df_players, combined_pdf)
        if ideal_path is None:
            path_deviation = 0
        else:
            start = (bc['y'].values[0], bc['x'].values[0])
            end = (np.round(start[0] + bc['xSpeed'].values[0] * 0.5, 3), np.round(start[1] + bc['ySpeed'].values[0] * 0.5, 3))
            actual_path = [start, end]
            path_deviation = directed_hausdorff(actual_path, ideal_path)[0]

    individual_defensive_pressures = [0] * 23
    index = 0
    for pdf in defenders_pdf:
        individual_defensive_pressure = np.sum(pdf * bc_pdf) / 5
        individual_defensive_pressures[index] = individual_defensive_pressure
        index += 1
    total_defensive_pressures = [np.sum(defensive_pdf * bc_pdf) / 5] * 23
    total_pressures = [np.sum(pressure_pdf) / 5] * 23
    path_deviations = [path_deviation] * 23

    modified_df = df[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'time', 'jerseyNumber', 'club', 'event']].copy()
    modified_df['individual_defensive_pressure'] = individual_defensive_pressures
    modified_df['total_defensive_pressures'] = total_defensive_pressures
    modified_df['total_pressures'] = total_pressures
    modified_df['path_deviations'] = path_deviations

    return modified_df



def process_play(df1, df2, df3, df4, df5, playId, gameId):
    distinct_frames = list(df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId), 'frameId'].unique())
    try:
        start_frame = df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId) & ((df1['event'] == 'handoff') | (df1['event'] == 'pass_outcome_caught')), 'frameId'].values[0]
    except:
        start_frame = 0
    end_frame = df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId) & ((df1['event'] == 'tackle') | (df1['event'] == 'fumble') | (df1['event'] == 'touchdown') | (df1['event'] == 'out_of_bounds')), 'frameId'].values[0]
    frame_dfs = []

    for frameId in distinct_frames:
        df = processToVisualizeFrame(df1, df2, df3, df4, df5, gameId, playId, frameId).copy()
        mod_df = process_frame(df, start_frame)
        frame_dfs.append(mod_df)
        print(f"Finished processing frame{frameId}   Progress({1+distinct_frames.index(frameId)}/{len(distinct_frames)})")

    play_df = pd.concat(frame_dfs, axis=0, ignore_index=True)
    play_df['individual_pressure_percentage'] = play_df['individual_defensive_pressure'] / play_df['total_defensive_pressures']
    play_df['individual_pressure'] = play_df['individual_pressure_percentage'] * play_df['total_pressures']
    play_df['generated_path_deviation'] = play_df['individual_pressure_percentage'] * play_df['path_deviations']

    zero_total_pressures = play_df['total_pressures'] < 0
    columns_to_zero = ['individual_pressure_percentage','individual_pressure','generated_path_deviation']
    play_df.loc[zero_total_pressures, columns_to_zero] = 0

    play_df['start_frame'] = start_frame
    play_df['end_frame'] = end_frame

    return play_df



def process_game(df1, df2, df3, df4, df5, gameId):
    game =  df1[df1['gameId']==gameId]
    distinct_plays = list(game.playId.unique())

    for broken in game[game.event == 'lateral'].playId.unique():
        distinct_plays.remove(broken)

    play_dfs = []

    for play in distinct_plays[37:]:
        play_df = process_play(df1, df2, df3, df4, df5, play, gameId)
        play_dfs.append(play_df)
        print(f"Finished processing play{play}   Progress({1+distinct_plays.index(play)}/{len(distinct_plays)})")
        break
    game_df = pd.concat(play_dfs, axis=0, ignore_index=True)


    return game_df


In [ ]:
#create_football_field Function
#Author: Hassaan Inayatali
#imgSize parameter determines how large you want the football field to be sized
#playerCoordinatesProvided allows you to specify whether or not you want to include player coordinates in the field visualization
#playerCoordinates includes the table of data with the associated players
#labelNumbers specifies whether you want the player numbers to be on the visualization
#showArrow specifies whether you want the direction of the player to be shown on the viz
#fieldColor and endZoneColor specifies the color of each. Note the endzone color will be a blend of the field color and the endZoneColor
# ZORDER
# field:0, Patches:1, Contour:2, Tessalation:3, Arrows:4, Players:5, Numbers:6


def create_football_field(imgSize=(12.66, 24), playerCoordinatesProvided=False, playerCoordinates=[],
                          labelNumbers=True, showArrow=False,
                          fieldColor='green', endZoneColor='black',
                          ball_carrier=False, offense=False, defense=False, combined=False, pressure=False,
                          start_frame=0):
    fig, ax = plt.subplots(1, figsize=imgSize)

    # Field
    rect = patches.Rectangle((0, 0), 53.3, 120, linewidth=0.1, edgecolor='r', facecolor=fieldColor, zorder=0)
    plt.plot([0.0, 53.3, 53.3, 0.0, 0.0, 53.3, 53.3, 0.0, 0.0, 53.3, 53.3, 0.0,
              0.0, 53.3, 53.3, 0.0, 0.0, 53.3, 53.3, 0.0, 0.0, 53.3, 53.3, 0.0],
             [10.0, 10.0, 20.0, 20.0, 30.0, 30.0, 40.0, 40.0, 50.0, 50.0, 60.0, 60.0,
              70.0, 70.0, 80.0, 80.0, 90.0, 90.0, 100 , 100 , 110 , 110 , 120 , 120 ],
             color='white')
    homeEndzone = patches.Rectangle((0, 0), 53.3, 10, linewidth=0.1, edgecolor='r', facecolor=endZoneColor, alpha=0.2, zorder=1)
    awayEndzone = patches.Rectangle((0, 110), 53.3, 10, linewidth=0.1, edgecolor='r', facecolor=endZoneColor, alpha=0.2, zorder=1)
    ax.add_patch(homeEndzone), ax.add_patch(awayEndzone)


    # Field Numbers and Hash Lines
    for y in range(20, 110, 10):
        number = y if y <= 50 else 120 - y
        plt.text(0.00 + 5, y - 0.9, str(number - 10), horizontalalignment='center', fontsize=20, color='white', rotation=270)
        plt.text(53.3 - 5, y - 0.5, str(number - 10), horizontalalignment='center', fontsize=20, color='white', rotation=90)
    for y in range(11, 110):
        ax.plot([0.00 + 0.5, 0.00 + 1.0], [y, y], color='white')
        ax.plot([53.3 - 1.0, 53.3 - 0.5], [y, y], color='white')
        ax.plot([17.4 + 0.5, 17.4 + 1.0], [y, y], color='white')
        ax.plot([35.9 - 1.0, 35.9 - 0.5], [y, y],  color='white')

    # Players and Football
    if playerCoordinatesProvided:
        for index, row in playerCoordinates.iterrows():
            color = 'red' if row['club'] == row['homeTeamAbbr'] else 'blue' if row['club'] == row['visitorTeamAbbr'] else 'brown'
            marker_size = 250 if color in ('red', 'blue') else 125
            if color != 'brown':
                plt.scatter(row['y'], row['x'], color=color, marker='o', s=marker_size, edgecolor='none', zorder=5)
                if showArrow:
                    arrow_color = 'orange' if color == 'red' else 'aquamarine'
                    plt.arrow(row['y'], row['x'], row['xSpeed'], row['ySpeed'], color=arrow_color, width=0.1, zorder=4)
                if labelNumbers:
                    plt.annotate(int(row['jerseyNumber']), (row['y'], row['x']), xytext=(row['y'], row['x'] - 0.35), ha='center', color='white', zorder=6)
            else:
                plt.scatter(row['y'], row['x'], color=color, s=marker_size, zorder=5)

    # PDFs and Ideal Path
    pdf_map = {}
    pressure_pdf = None
    pursuit_cushion = 2

    if playerCoordinatesProvided:
        df = playerCoordinates
        bc = df.loc[df['displayName'] == df['ballCarrierDisplayName']]
        df_players = df.loc[(bc['x'].values[0] + pursuit_cushion > (df['x']))]

        current_frame = df.frameId.values[0]

        x, y = np.mgrid[0:53.3:0.1, 0:120:0.1]
        locations = np.dstack((x, y))

        bc_pdf = create_bc_pdf(bc, locations)

        # defensive_pdf, offensive_pdf = create_pdf(df_players, locations, bc, graph=True)
        defensive_pdf, offensive_pdf = create_pdf(df, locations, bc, graph=True)


        combined_pdf = np.add(offensive_pdf, defensive_pdf)

        pressure_pdf = combined_pdf * bc_pdf
        pressure = np.round(np.sum(pressure_pdf) / 5, 3)

        # if ball_carrier:
        #     graph_pdf(ax, x, y, bc_pdf, 'bc')
        # if offense:
        #     graph_pdf(ax, x, y, offensive_pdf, 'offense')
        # if defense:
        #     graph_pdf(ax, x, y, defensive_pdf, 'defense')
        if combined:
            graph_pdf(ax, x, y, combined_pdf, 'combined')
        # if pressure:
        #     graph_pdf(ax, x, y, pressure_pdf, 'pressure')

        path_deviation = 0

        if current_frame >= start_frame:
            # ideal_path = play(df, combined_pdf)
            ideal_path = process(df, combined_pdf)
            if ideal_path is not None:
                if len(ideal_path) == 2:
                    x_values = [coord[0] for coord in ideal_path]
                    y_values = [coord[1] for coord in ideal_path]
                else:
                    for edge in ideal_path:
                        x_values = [coord[0] for coord in edge]
                        y_values = [coord[1] for coord in edge]
            if ideal_path is None:
                path_deviation = 0
            else:
                start = (bc['y'].values[0], bc['x'].values[0])
                end = (np.round(start[0] + bc['xSpeed'].values[0] * 0.5, 3), np.round(start[1] + bc['ySpeed'].values[0] * 0.5, 3))
                actual_path = [start, end]
                path_deviation = np.round(directed_hausdorff(actual_path, ideal_path)[0], 3)

        stat_display = patches.Rectangle((0, -6), 53.3, 6, linewidth=0.1, edgecolor='r', facecolor='gray', zorder=0)
        ax.text(5, 5, f'Pressure: {pressure}', horizontalalignment='left', fontsize=28, color='white')
        ax.text(25, 5, f'Path Deviation: {path_deviation}', horizontalalignment='left', fontsize=28, color='white')
        ax.add_patch(stat_display)
        ax.text(54.3/2, -3.5, f'Example play: -4 yard run', horizontalalignment='center', fontsize=28, color='white')

    ax.add_patch(rect)
    plt.ylim(-6, 120)
    plt.xlim(-1, 54.3)
    plt.axis('off')

    return pressure

In [ ]:
def extractAllImagesForAPlay(df1, df2, df3, df4, df5, playId, gameId):
    distinctTimes = df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId), 'time'].unique() #Extracts all times associated with a particular play

    array_of_images = []
    directory = f"Play_{playId}"
    parent_dir = "/content/"
    path = os.path.join(parent_dir, directory)

    try:
        os.mkdir(path)
    except FileExistsError:
        pass

    os.chdir(path)
    try:
        start_frame = df1.loc[(df1['playId'] == playId) & (df1['gameId'] == gameId) & ((df1['event'] == 'handoff') | (df1['event'] == 'pass_outcome_caught')), 'frameId'].values[0]
    except:
        start_frame = 0
    for i in distinctTimes:
        df_for_running = processToVisualizeFrame(df1, df2, df3, df4, df5, playId, i, gameId)
        create_football_field(playerCoordinatesProvided=True, playerCoordinates=df_for_running, combined=True, start_frame=start_frame)
        plt.savefig(f'imgTime:{i}.png') #saves image into the folder
        array_of_images.append(f'imgTime:{i}.png') #creates a list of the image names
        plt.close()

    files = [imageio.imread(filename) for filename in array_of_images]  #appends the image to the files
    imageio.mimsave(f'play{playId}.gif', files)  #Generates the gif of the play
    os.chdir('..')


# 3 Yard Run week1 2022091101 2320
# -4 Yard Run week4 2022100201 716
# 15 Yard Run week1 2022091100	1608
gameId =  2022100201
playId = 716
week = week4

extractAllImagesForAPlay(week, games, plays, players, tackles, playId, gameId)

In [ ]:
# game[['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'time', 'jerseyNumber', 'club', 'event']]

# ADD FOLLOWING COLUMNS PRESSURE EXERTERTED, % OF TEAM PRESSURE, TOTAL PRESSURE EXERTED, PATH DEVIATION

In [ ]:
# array([2022090800, 2022091100, 2022091101, 2022091102, 2022091103,
#       2022091104, 2022091105, 2022091106, 2022091107, 2022091108,
#       2022091109, 2022091110, 2022091111, 2022091112, 2022091113,
#       2022091200])

week = "week4"
gameId = 2022092900
# gameId = 2022091100
test = process_game(week4, games, plays, players, tackles, gameId)

test.to_csv(f"{week}_game{gameId}.csv", index=False)
files.download(f"{week}_game{gameId}.csv")

In [ ]:
# ####################################################################################################
#     #
#     ### Parameters
#     #   1) df_frame
#     ### Return
#     #   1) triangle                               Delaunay
#     #   2) triangle_edges                         set[tuple()]
#     #   3) sideline_edges
#     #   4) sideline_points
#     #   5) all_points
# ############################################################################################################################## FINISH ANNOTATING
# def generate_sideline_windows(df, points):
#     sideline_edges = []
#     sideline_points = []
#     try:
#         hull = ConvexHull(df)
#         for vertex in hull.vertices:
#             coordinates = points[vertex]
#             no_intersect_left, no_intersect_right = True, True
#             zero_segment, nzero_segment = (0, coordinates[1]), (53.3, coordinates[1])
#             for simplex in hull.simplices:
#                 probe_ls = LineString([points[simplex[1]], points[simplex[0]]])
#                 zero_segment_ls = LineString([points[vertex], zero_segment])
#                 nzero_segment_ls = LineString([points[vertex], nzero_segment])
#                 if vertex not in simplex:
#                     if no_intersect_left and zero_segment_ls.intersects(probe_ls):
#                         no_intersect_left = False
#                     if no_intersect_right and nzero_segment_ls.intersects(probe_ls):
#                         no_intersect_right = False
#             if no_intersect_left:
#                 sideline_edges.append([zero_segment, points[vertex]])
#                 sideline_points.append(zero_segment)
#             if no_intersect_right:
#                 sideline_edges.append([nzero_segment, points[vertex]])
#                 sideline_points.append(nzero_segment)
#     except:
#         for triangle_point in points:
#             zero_segment, nzero_segment = (0, triangle_point[1]), (53.3, triangle_point[1])
#             sideline_edges.append([nzero_segment, triangle_point])
#             sideline_edges.append([zero_segment, triangle_point])
#             sideline_points.append(nzero_segment)
#             sideline_points.append(zero_segment)
#     sideline_edges.append([(0,10),(53.3,10)])
#     sideline_points.extend([(0,10),(53.3,10)])
#     return sideline_edges, sideline_points

# ####################################################################################################
#     #
#     ### Parameters
#     #   1) df_frame
#     ### Return
#     #   1) triangle                               Delaunay
#     #   2) triangle_edges                         set[tuple()]
#     #   3) sideline_edges
#     #   4) sideline_points
#     #   5) all_points
# ############################################################################################################################## FINISH ANNOTATING
# # def generate_prerequisites(df_frame):
# def generate_prerequisites(df_frame):
#     triangle_points = [tuple(arr) for arr in df_frame[['y', 'x']].values]
#     try:
#         triangle = Delaunay(triangle_points)
#     except:
#         triangle = None

#     sideline_edges, sideline_points = generate_sideline_windows(df_frame[['y', 'x']], triangle_points)

#     all_points = triangle_points + sideline_points

#     return triangle, triangle_points, sideline_edges, sideline_points, all_points


# ''' Finds all unique simplicies for the triangle
#     Parameters
#     1) triangle                                   Delaunay
#     Return
#     1) triangle_edges                             set[tuple()]
# '''
# def create_Unique_Triangle_Simplices(triangle) -> set:
#     triangle_edges = set()
#     try:
#         for simplex in triangle.simplices:
#             edges = [(min(simplex[i], simplex[(i + 1) % 3]), max(simplex[i], simplex[(i + 1) % 3])) for i in range(3)]
#             triangle_edges.update(edges)
#     except AttributeError:
#         triangle_edges = set()  # Returning an empty set in case of an AttributeError
#     return triangle_edges

# ''' Creates TWO MAPS: {point: coordinates} & {coordinates: points}
#     Parameters
#     1) points                                     list[tuple()]
#     Return
#     1) point_to_coordinates                       dict[int: tuple(float,float)]
#     2) coordinatess_to_point                      dict[tuple(float,float): int]
# '''
# def create_Point_and_Coordinates_Dictionaries(all_points) -> dict:
#     point_to_coordinates = {index: coordinates for index, coordinates in enumerate(all_points)}
#     coordinates_to_point = {coordinates: index for index, coordinates in enumerate(all_points)}
#     return point_to_coordinates, coordinates_to_point


# ''' Creates TWO MAPS: {edge: endpoints} & {endpoints: edge}
#     Parameters
#         1) triangle_edges                         set[tuple()]
#         2) sideline_edges                         list[list[tuple(float,float)]]
#         3) coordinates_to_point                   dict[tuple(float,float): int]
#     Return
#         1) edge_to_endpoints                      dict[int: tuple(int,int)]
#         2) endpoints_to_edge                      dict[tuple(int,int): int]
# '''
# def create_Edge_and_Endpoints_Dictionaries(triangle_edges, sideline_edges, coordinates_to_point) -> dict:
#     edge_to_endpoints = {}
#     endpoints_to_edge = {}

#     for index, simplex in enumerate(triangle_edges):
#         endpoints_to_edge[simplex] = index
#         edge_to_endpoints[index] = simplex

#     start_index = len(triangle_edges)
#     for index, edge in enumerate(sideline_edges, start=start_index):
#         simplex = (coordinates_to_point[edge[1]], coordinates_to_point[edge[0]])
#         endpoints_to_edge[simplex] = index
#         edge_to_endpoints[index] = simplex

#     return edge_to_endpoints, endpoints_to_edge


# ####################################################################################################
#     # Creates Dictionary that maps Current Edge to its Valid Neighboring Edges
#     ### Parameters
#     #   1) triangle                               Delaunay
#     #   2) triangle_points
#     #   3) triangle_edges                         set[tuple()]
#     #   4) sideline_edges                         list[list[tuple(float,float)]]
#     #   5) sideline_points
#     #   6) point_to_coordinates                   dict[int: tuple(float,float)]
#     #   7) coordinates_to_point                   dict[tuple(float,float): int]
#     #   8) edge_to_endpoints                      dict[int: tuple(int,int)]
#     #   9) endpoints_to_edge                      dict[tuple(int,int): int]
#     ### Return
#     #   1) edge_to_neighboring_edges
# ###################################################################################################################################         ANNOTATE AT THE END
# def create_Edge_to_Neighbors_Dictionary(triangle, triangle_points, triangle_edges, sideline_edges, sideline_points, point_to_coordinates, coordinates_to_point, edge_to_endpoints, endpoints_to_edge):
#     edge_to_neighboring_edges = {}
#     index = 0
#     ########################################
#     # Adds to Dictionary: Triangle Edges
#     ########################################
#     try:
#         hull_simplices = [tuple(sorted(arr)) for arr in ConvexHull(triangle_points).simplices]
#         for simplex in triangle_edges:
#             edges = []
#             for tri in triangle.simplices:
#                 y_limit = max(point_to_coordinates[simplex[0]][1],point_to_coordinates[simplex[1]][1])
#                 if simplex[0] in tri and simplex[1] in tri:
#                     third_vertex = list(tri.copy())
#                     third_vertex.remove(simplex[0])
#                     third_vertex.remove(simplex[1])
#                     third_vertex = third_vertex[0]
#                     if point_to_coordinates[third_vertex][1] < y_limit:
#                         edge_1 = endpoints_to_edge[min(third_vertex, simplex[0]), max(third_vertex, simplex[0])]
#                         edge_2 = endpoints_to_edge[min(third_vertex, simplex[1]), max(third_vertex, simplex[1])]
#                         edges.extend([edge_1, edge_2])
#             if simplex in hull_simplices:
#                 endpoint_1 = point_to_coordinates[simplex[0]]
#                 endpoint_2 = point_to_coordinates[simplex[1]]
#                 if endpoint_1[1] < endpoint_2[1]:
#                     if (0, endpoint_1[1]) in sideline_points:
#                         edges.append(endpoints_to_edge[simplex[0], coordinates_to_point[0, endpoint_1[1]]])
#                     if (53.3, endpoint_1[1]) in sideline_points:
#                         edges.append(endpoints_to_edge[simplex[0], coordinates_to_point[53.3, endpoint_1[1]]])
#                 else:
#                     if (0, endpoint_2[1]) in sideline_points:
#                         edges.append(endpoints_to_edge[simplex[1], coordinates_to_point[0, endpoint_2[1]]])
#                     if (53.3, endpoint_1[1]) in sideline_points:
#                         edges.append(endpoints_to_edge[simplex[0], coordinates_to_point[53.3, endpoint_1[1]]])
#             edge_to_neighboring_edges[index] = edges
#             index += 1
#     except:
#         pass
#     ########################################
#     # Adds to Dictionary: Sideline Edges
#     ########################################
#     for sideline_coordinates in sideline_edges:
#         if sideline_coordinates == [(0, 10), (53.3, 10)]:
#             continue
#         edges = []
#         sideline = endpoints_to_edge[(coordinates_to_point[sideline_coordinates[1]], coordinates_to_point[sideline_coordinates[0]])]
#         try:
#             for simplex in hull_simplices:
#                 if coordinates_to_point[sideline_coordinates[1]] in simplex:
#                     endpoint_1 = point_to_coordinates[simplex[0]]
#                     endpoint_2 = point_to_coordinates[simplex[1]]
#                     if (endpoint_1[1] <= sideline_coordinates[1][1]) and \
#                     (endpoint_2[1] <= sideline_coordinates[1][1]):
#                         edges.append(endpoints_to_edge[simplex])
#         except:
#             pass
#         if len(edges) > 1:
#             edge_1 = edge_to_endpoints[edges[0]]
#             edge_2 = edge_to_endpoints[edges[1]]
#             if sideline_coordinates[0][0] == 0:
#                 if edge_1[0] == edge_2[0]:
#                     if point_to_coordinates[edge_1[1]][0] < point_to_coordinates[edge_2[1]][0]:
#                         edges.remove(edges[1])
#                     else:
#                         edges.remove(edges[0])
#                 else:
#                     if point_to_coordinates[edge_1[0]][0] < point_to_coordinates[edge_2[0]][0]:
#                         edges.remove(edges[1])
#                     else:
#                         edges.remove(edges[0])
#             else:
#                 if edge_1[0] == edge_2[0]:
#                     if point_to_coordinates[edge_1[1]][0] > point_to_coordinates[edge_2[1]][0]:
#                         edges.remove(edges[1])
#                     else:
#                         edges.remove(edges[0])
#                 else:
#                     if point_to_coordinates[edge_1[0]][0] > point_to_coordinates[edge_2[0]][0]:
#                         edges.remove(edges[1])
#                     else:
#                         edges.remove(edges[0])
#         sorted_sideline_edges = sorted(sideline_edges, key=lambda x: x[0][1], reverse=True)
#         for i in range(sorted_sideline_edges.index(sideline_coordinates) + 1, len(sorted_sideline_edges)):
#             if i == len(sorted_sideline_edges) - 1:
#                 endpoint_1 = coordinates_to_point[sorted_sideline_edges[i][1]]
#                 endpoint_2 = coordinates_to_point[sorted_sideline_edges[i][0]]
#                 edges.append(endpoints_to_edge[(endpoint_1, endpoint_2)])
#                 break
#             if sorted_sideline_edges[i][0][0] == sideline_coordinates[0][0]:
#                 endpoint_1 = coordinates_to_point[sorted_sideline_edges[i][1]]
#                 endpoint_2 = coordinates_to_point[sorted_sideline_edges[i][0]]
#                 edges.append(endpoints_to_edge[(endpoint_1, endpoint_2)])
#                 break
#         edge_to_neighboring_edges[index] = edges
#         index += 1
#     return edge_to_neighboring_edges

# ####################################################################################################
#     # Creates Dictionary that maps Current Edge to its Current Nodes
#     ### Parameters
#     #   1) point_to_coordinates                   dict[int: tuple(float,float)]
#     #   2) edge_to_endpoints                      dict[int: tuple(int,int)]
#     ### Return
#     #   1) edges_to_nodes
#     #   2) positions
#     #   3) endzones_nodes
# ###################################################################################################################################         ANNOTATE AT THE END
# def create_Edge_to_Nodes_Dictionary(point_to_coordinates, edge_to_endpoints, start) -> tuple:
#     edges_to_current_nodes = {}
#     endzones_nodes = []
#     positions = {}
#     for edge, endpoints in edge_to_endpoints.items():
#         endpoint_1 = point_to_coordinates[endpoints[0]]
#         endpoint_2 = point_to_coordinates[endpoints[1]]
#         edge_length = np.round(((endpoint_1[0]-endpoint_2[0])**2 + (endpoint_1[1]-endpoint_2[1])**2)**0.5, 2)

#         if edge_length < 1:
#             divisions = 2
#         elif 1 <= edge_length < 2.5:
#             divisions = 5
#         elif 2.5 <= edge_length < 5:
#             divisions = 10
#         elif 5 <= edge_length <= 20:
#             divisions = 10
#         else:
#             divisions = 50

#         x_values = np.linspace(endpoint_1[0], endpoint_2[0], divisions, endpoint=False)
#         y_values = np.linspace(endpoint_1[1], endpoint_2[1], divisions, endpoint=False)

#         nodes = [(np.round(x_values[i], 3), np.round(y_values[i], 3)) for i in range(1, divisions)]
#         edges_to_current_nodes[edge] = nodes

#         if (endpoint_1 in [(0, 10), (53.3, 10)]) and (endpoint_2 in [(0, 10), (53.3, 10)]):
#             endzones_nodes = nodes

#     positions[start] = start

#     return edges_to_current_nodes, positions, endzones_nodes



# ####################################################################################################
#     # Creates Closest Edges
#     ### Parameters
#     #   1) sideline_edges
#     #   2) coordinates_to_point                   dict[tuple(float,float): int]
#     #   3) endpoints_to_edge                      dict[tuple(int,int): int]
#     ### Return
#     #   1) closest_edges
# ###################################################################################################################################         ANNOTATE AT THE END
# def create_Closest_Edges(sideline_edges, coordinates_to_point, endpoints_to_edge):
#     if len(coordinates_to_point) <= 2:
#         closest_edges = [0]
#     else:
#         closest_edges = []
#         closest_edge_value = max(sum(coordinate[1] for coordinate in sideline) for sideline in sideline_edges)
#         closest_edges_coordinates = [sideline for sideline in sideline_edges if sum(coordinate[1] for coordinate in sideline) == closest_edge_value]
#         closest_edges = [endpoints_to_edge[coordinates_to_point[closest_edges_coordinates[0][1]], coordinates_to_point[closest_edges_coordinates[0][0]]],
#                         endpoints_to_edge[coordinates_to_point[closest_edges_coordinates[1][1]], coordinates_to_point[closest_edges_coordinates[1][0]]]]
#     return closest_edges

# ####################################################################################################
#     # Creates Path Graph
#     ### Parameters
#     #   1) start
#     #   2) edge_to_neighboring_edges
#     #   3) edge_to_current_nodes
#     #   4) closest_edges
#     ### Return
#     #   1) graph
# ###################################################################################################################################         ANNOTATE AT THE END ############### ADD PRESSURE WEGIHTINGS
# # def calculate_path_weights(pdf, start, path_end):
# #     x_over_y = (path_end[0] - start[0]) / (path_end[1] - start[1])
# #     iterations = abs(round(path_end[1] - start[1] * 10))
# #     path_weights = []
# #     for index in range(iterations):
# #         x_coordinate = start[0] * 10 - index * x_over_y
# #         y_coordinate = round(start[1] * 10) - index
# #         path_weights.append(sum(pdf[math.floor(x_coordinate) - 10:math.ceil(x_coordinate) + 10][:, y_coordinate]))
# #     return sum(path_weights) / 5

# def calculate_path_weights(pdf, start, path_end):
#     x_over_y = (path_end[0] - start[0]) / (path_end[1] - start[1])
#     iterations = abs(round(path_end[1] - start[1] * 10))
#     path_weights = []

#     # Precompute x and y coordinates for the entire range
#     x_coordinates = np.array([start[0] * 10 - index * x_over_y for index in range(iterations)])
#     y_coordinates = np.round(start[1] * 10) - np.arange(iterations)

#     # Use vectorized operations to calculate weights
#     for index in range(iterations):
#         x_coord = x_coordinates[index]
#         y_coord = y_coordinates[index]

#         x_floor, x_ceil = int(np.floor(x_coord)) - 10, int(np.ceil(x_coord)) + 10
#         y_val = int(y_coord)

#         path_weights.append(np.sum(pdf[x_floor:x_ceil, y_val]))

#     return np.sum(path_weights) / 5


# def adjust_distance(distance, path_weight, start, end):
#     adjusted_distance = distance * expit(path_weight)
#     return float(adjusted_distance)

# #################### VERSION 1 (Distance + Path Weight | Closest Edge)
# #################### VERSION 2 (Distance | A STAR)
# #################### VERSION 3 (Distance + Path Weight | A STAR)
# def create_Graph(start, edge_to_neighboring_edges, edge_to_current_nodes, closest_edges, pdf, version=1):
#     if version == 1:
#         graph = {}
#         print(closest_edges)
#         for edge in closest_edges:
#             for path_end in edge_to_current_nodes[edge]:
#                 if start[1] > path_end[1]:
#                     distance = ((start[0] - path_end[0])**2 + (start[1] - path_end[1])**2)**0.5
#                     path_weight = calculate_path_weights(pdf, start, path_end) / distance
#                     adjusted_distance = adjust_distance(distance, path_weight, start, path_end)
#                     graph[adjusted_distance] = [start, path_end]
#     elif version == 2:
#         graph = nx.Graph()

#         for edge in closest_edges:
#             for path_end in edge_to_current_nodes[edge]:
#                 if start[1] > path_end[1]:
#                     distance = ((start[0] - path_end[0])**2 + (start[1] - path_end[1])**2)**0.5
#                     graph.add_edge(start, path_end, weight=distance)

#         for edge, next_edges in edge_to_neighboring_edges.items():
#             for path_start in edge_to_current_nodes[edge]:
#                 for next_edge in next_edges:
#                     for path_end in edge_to_current_nodes[next_edge]:
#                         if path_start[1] > path_end[1]:
#                             distance = ((path_start[0] - path_end[0])**2 + (path_start[1] - path_end[1])**2)**0.5
#                             graph.add_edge(path_start, path_end, weight=distance)
#     elif version == 3:
#         graph = nx.Graph()

#         for edge in closest_edges:
#             for path_end in edge_to_current_nodes[edge]:
#                 if start[1] > path_end[1]:
#                     distance = ((start[0] - path_end[0])**2 + (start[1] - path_end[1])**2)**0.5
#                     path_weight = calculate_path_weights(pdf, start, path_end) / distance
#                     adjusted_distance = adjust_distance(distance, path_weight, start, path_end)
#                     graph.add_edge(start, path_end, weight=adjusted_distance)

#         for edge, next_edges in edge_to_neighboring_edges.items():
#             for path_start in edge_to_current_nodes[edge]:
#                 for next_edge in next_edges:
#                     for path_end in edge_to_current_nodes[next_edge]:
#                         if path_start[1] > path_end[1]:
#                             distance = ((path_start[0] - path_end[0])**2 + (path_start[1] - path_end[1])**2)**0.5
#                             path_weight = calculate_path_weights(pdf, path_start, path_end)
#                             adjusted_distance = adjust_distance(distance, path_weight, start, path_start)
#                             graph.add_edge(path_start, path_end, weight=adjusted_distance)
#     return graph

# ####################################################################################################
#     # Creates Path Graph
#     ### Parameters
#     #   1) start                                  INDEX
#     #   2) graph
#     #   3) edge_to_current_nodes
#     #   4) closest_edges
#     ### Return
#     #   1) ideal_path_edges

#     #   VERSION 1 (Shortest Path)
#     #   VERSION 2 (A Star)
# ###################################################################################################################################         ANNOTATE AT THE END ############### ADD PRESSURE WEGIHTINGS
# def get_Ideal_Path(start, graph, endzone_nodes, version=1):
#     if version == 1:
#         best_cost = min(list(graph.keys()))
#         best_path_edges = graph[best_cost]
#     elif version == 2:
#         best_path_cost = float('inf')
#         best_path = None
#         filtered_endzone_nodes = [node for node in endzone_nodes if abs(node[0] - start[0]) <= 10]
#         for endzone_node in filtered_endzone_nodes:
#             try:
#                 cost = nx.astar_path_length(graph, start, endzone_node, weight='weight')
#                 if cost < best_path_cost:
#                     best_path_cost = cost
#                     best_path = nx.astar_path(graph, start, endzone_node, weight='weight')
#             except nx.NetworkXNoPath:
#                 continue
#         try:
#             best_path_edges = list(zip(best_path[:-1], best_path[1:]))
#         except (TypeError, AttributeError):
#             best_path_edges = None

#     return best_path_edges


# ####################################################################################################
#     # Graphs Windows
#     ### Parameters
#     #   1) df                                     DataFrame
#     #   2) triangle                               Delaunay
#     #   3) sideline_edges
#     ### Return
# ###################################################################################################################################
# def graph_Windows(df, triangle, sideline_edges):
#     triangle_points_graphing = df[['y', 'x']].values
#     try:
#         plt.triplot(triangle_points_graphing[:, 0], triangle_points_graphing[:, 1], triangle.simplices,
#                     markersize=1, color='black',
#                     linestyle='dashed', linewidth=1, zorder=3)
#     except AttributeError:
#         pass

#     for simplex in sideline_edges:
#         x, y = zip(*simplex)
#         plt.plot(x, y, markersize=1, color='brown', linestyle='dashed', linewidth=1, zorder=3)


# def create_bc_pdf(df, locations):
#     # Rotation Matrix
#     xComponent, yComponent = df['xComponent'].values[0], df['yComponent'].values[0]
#     r_Matrix = np.array([[xComponent, -yComponent], [yComponent, xComponent]])

#     # Scaling Matrix
#     speed_Ratio = (df['s'].values[0] ** 2) / (13.25 ** 2)
#     bc_x = df['y'].values[0] + df['xSpeed'].values[0] * 0.5
#     bc_y = df['x'].values[0] + df['ySpeed'].values[0] * 0.5

#     dist = 2
#     topLeftSMatrix = (dist + dist * speed_Ratio) * 0.5
#     botRightSMatrix = (dist - dist * speed_Ratio) * 0.5
#     s_Matrix = np.array([[topLeftSMatrix + 0.000001, 0], [0, botRightSMatrix - 0.000001]])

#     # Covariance Matrix Calculations
#     covariance_matrix = r_Matrix @ s_Matrix @ s_Matrix.T @ np.linalg.inv(r_Matrix)

#     # Mu Values
#     mu_val_x = df['y'].values[0] + df['xSpeed'].values[0] * 0.2
#     mu_val_y = df['x'].values[0] + df['ySpeed'].values[0] * 0.2
#     bc_mu = np.array([mu_val_x, mu_val_y])

#     # Creating BallCarrier PDF
#     bc_pdf = stats.multivariate_normal.pdf(locations, mean=bc_mu, cov=covariance_matrix)

#     return bc_pdf

# def create_pdf(df, locations, bc, graph=False):
#     defensive_pdf, offensive_pdf = 0.0, 0.0
#     defenders_pdfs = []

#     for index, row in df.iterrows():
#         if row['displayName'] == row['ballCarrierDisplayName'] or row['displayName'] == 'football':
#             continue

#         # Rotation Matrix
#         xComponent, yComponent = row['xComponent'], row['yComponent']
#         r_Matrix = np.array([[xComponent, -yComponent], [yComponent, xComponent]])
#         i_r_Matrix = np.linalg.inv(r_Matrix)

#         # Scaling Matrix
#         speed_Ratio = (row['s'] ** 2) / (13.25 ** 2)
#         topLeftSMatrix = 0.5 + (row['dist_to_bc'] + row['dist_to_bc'] * speed_Ratio) * 0.5
#         botRightSMatrix = 0.5 + (row['dist_to_bc'] - row['dist_to_bc'] * speed_Ratio) * 0.5
#         s_Matrix = np.array([[topLeftSMatrix + 0.000001, 0], [0, botRightSMatrix - 0.000001]])

#         # Covariance Matrix Calculations
#         rs_Matrix = np.dot(r_Matrix, s_Matrix)
#         rss_Matrix = np.dot(rs_Matrix, s_Matrix)
#         covariance_Matrix = np.dot(rss_Matrix, i_r_Matrix)

#         # Mu Values
#         mu_val_x = row['y'] + row['xSpeed'] * 0.5
#         mu_val_y = row['x'] + row['ySpeed'] * 0.5
#         mu = [mu_val_x, mu_val_y]

#         player_pdf = stats.multivariate_normal(mu, covariance_Matrix).pdf(locations)

#         if row['club'] == row['defensiveTeam']:
#             if not graph:
#                 defenders_pdfs.append(player_pdf)
#             defensive_pdf += player_pdf
#         elif row['club'] == row['possessionTeam']:
#             closest_tackler_distance = min(df.loc[df['club'] == df['defensiveTeam']]['dist_to_bc'])

#             closest_tackler_distance_blocker = 13105
#             yCoordinatePassRusher = 0
#             xCoordinatePassRusher = 0
#             speedPassRusher = 0

#             df_defensive = df.loc[df['club'] == df['defensiveTeam']]
#             for _, defense_row in df_defensive.iterrows():
#                 distance = np.sqrt((defense_row['x'] - row['x']) ** 2 + (defense_row['y'] - row['y']) ** 2)
#                 if distance <= closest_tackler_distance_blocker:
#                     closest_tackler_distance_blocker = distance
#                     xCoordinatePassRusher = defense_row['x']
#                     yCoordinatePassRusher = defense_row['y']
#                     speedPassRusher = defense_row['s']

#             p12 = row['dist_to_bc']
#             p23 = np.sqrt((bc['x'].values[0] - xCoordinatePassRusher) ** 2 + (bc['y'].values[0] - yCoordinatePassRusher) ** 2)
#             p13 = np.sqrt((row['x'] - xCoordinatePassRusher) ** 2 + (row['y'] - yCoordinatePassRusher) ** 2)

#             angleBetweenThreePlayers = np.arccos(((p12 ** 2) + (p13 ** 2) - (p23 ** 2)) / (2 * p12 * p13))
#             degreesAngleBetweenThreePlayers = np.degrees(angleBetweenThreePlayers)
#             angleFrom180 = abs(degreesAngleBetweenThreePlayers - 180)

#             player_pdf = degreesAngleBetweenThreePlayers * (player_pdf / 180)
#             offensive_pdf -= player_pdf

#     if not graph:
#         return defenders_pdfs, defensive_pdf, offensive_pdf
#     else:
#         return defensive_pdf, offensive_pdf


# def play(df, pdf):
#     bc = df.loc[df['displayName'] == df['ballCarrierDisplayName']]
#     dfDefensive = df.loc[df['club']== df['defensiveTeam']]
#     in_play_players = dfDefensive.loc[(bc['x'].values[0] > (dfDefensive['x'])) & ((dfDefensive['x']) > 10)]

#     start = tuple(bc[['y','x']].values[0])

#     if start[1] <= 10:
#         return None

#     triangle, triangle_points, sideline_edges, sideline_points, all_points = generate_prerequisites(in_play_players)

#     ##### Graphing
#     graph_Windows(in_play_players, triangle, sideline_edges)

#     ##### Ideal Path
#     triangle_edges = create_Unique_Triangle_Simplices(triangle)
#     point_to_coordinates, coordinates_to_point = create_Point_and_Coordinates_Dictionaries(all_points)

#     edge_to_endpoints, endpoints_to_edge = create_Edge_and_Endpoints_Dictionaries(triangle_edges, sideline_edges, coordinates_to_point)
#     edge_to_neighboring_edges = create_Edge_to_Neighbors_Dictionary(triangle, triangle_points, triangle_edges, sideline_edges, sideline_points, point_to_coordinates, coordinates_to_point, edge_to_endpoints, endpoints_to_edge)
#     edge_to_current_nodes, positions, endzone_nodes = create_Edge_to_Nodes_Dictionary(point_to_coordinates, edge_to_endpoints, start)
#     closest_edges = create_Closest_Edges(sideline_edges, coordinates_to_point, endpoints_to_edge)
#     graph = create_Graph(start, edge_to_neighboring_edges, edge_to_current_nodes, closest_edges, pdf)
#     ideal_path = get_Ideal_Path(start, graph, endzone_nodes)
#     return ideal_path



In [ ]:
# RIGHT NOW 'start_frame':, 'end_frame'

# ['start_frame':, 'end_frame':,
#
#   a) Total Pressure Generated (In play)
#   b) Total Path Deviation (In play)
#   'in_total_pressure_generated':, 'in_total_path_deviation': ,

#   STEP 3
#   1) Normalized Total Pressure Generated (2 Factors [1) Play Result 2) Play Duration])
#   2) Normalized Total Path Deviation (2 Factors [1) Play Result 2) Play Duration])
### Result of the play is
### Duration of the play is

#   STEP 4
#   3) Change in Pressure (preplay -> peak) (inplay -> peak)    (Ability to get involved (Effort))
#   'pre_start_pressure':, 'in_start_pressure':, 'in_peak_pressure':, 'peak_pressure_frame':,


#   STEP 5
#   Miscellanious
#   4) Preplay Pressure Generated                   'pre_total_presure_generated':,
#   5) Ending Frames Pressure Generated             'post_total_presure_generated':,

### 5 yards deviation vs 10 vs 1




##### COLUMNS gameId	playId		club start_frame	end_frame	playDuration
##### displayName	 jerseyNumber    nflId   individual_pressure	generated_path_deviation